In [0]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("YourAppName") \
    .getOrCreate()

In [0]:
spark.conf.set("spark.databricks.io.cache.enabled", "false")
print("spark.databricks.io.cache.enabled is %s" % spark.conf.get("spark.databricks.io.cache.enabled"))

spark.databricks.io.cache.enabled is false


In [0]:
%pip install vaex

Python interpreter will be restarted.
  Created wheel for aplus: filename=aplus-0.11.0-py3-none-any.whl size=4401 sha256=c5fa892299f22bd079897801c874331fd33c1c6435be697ce35ff6c86aba34fb
  Stored in directory: /root/.cache/pip/wheels/e2/ec/10/f765aa7a52b46a9b836531727749cc586db038705bd885a0d5
Successfully built aplus
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.11.2
    Not uninstalling pygments at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-547e3444-2b8a-4421-bdba-f478a779c436
    Can't uninstall 'Pygments'. No files were found to uninstall.
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Not uninstalling packaging at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-547e3444-2b8a-4421-bdba-f478a779c436
    Can't uninstall 'packaging'. No files were found to uninstall.
  Attempting uninsta

In [0]:
import pandas as pd
import numpy as np
import vaex

print('pandas version: %s' % pd.__version__)

print('numpy version: %s' % np.__version__)

print('vaex version: %s' % vaex.__version__)

import pyarrow
print('pyarrow version: %s' % pyarrow.__version__)

import pyspark
print('pyspark version: %s' % pyspark.__version__)


import time

def benchmark(f, df, benchmarks, name, **kwargs):
    """Benchmark the given function against the given DataFrame.
    
    Parameters
    ----------
    f: function to benchmark
    df: data frame
    benchmarks: container for benchmark results
    name: task name
    
    Returns
    -------
    Duration (in seconds) of the given operation
    """
    start_time = time.time()
    ret = f(df, **kwargs)
    benchmarks['duration'].append(time.time() - start_time)
    benchmarks['task'].append(name)
    print(f"{name} took: {benchmarks['duration'][-1]} seconds")
    return benchmarks['duration'][-1]

def get_results(benchmarks):
    """Return a pandas DataFrame containing benchmark results."""
    return pd.DataFrame.from_dict(benchmarks)

pandas version: 2.2.2
numpy version: 1.26.4
vaex version: {'vaex': '4.17.0', 'vaex-core': '4.17.1', 'vaex-viz': '0.5.4', 'vaex-hdf5': '0.14.1', 'vaex-server': '0.9.0', 'vaex-astro': '0.9.3', 'vaex-jupyter': '0.8.2', 'vaex-ml': '0.18.3'}
pyarrow version: 7.0.0
pyspark version: 3.3.2.dev0


In [0]:
filenames = [f"/FileStore/tables/yellow_tripdata_2023_0{i}.parquet" for i in range(1, 6)]

dfs = []
for filename in filenames:
    # df = pd.read_parquet(filename)
    df = spark.read.format('parquet').options(header='true').load(filename).toPandas()

    if 'airport_fee' in df.columns:
        df.rename(columns={'airport_fee': 'Airport_fee'}, inplace=True)
    # df_dask = dd.from_pandas(df, npartitions=3)

    dfs.append(df)

pandas_data = pd.concat(dfs, ignore_index=True)
# dask_data = dd.concat(dfs)
vaex_data = vaex.from_pandas(pandas_data)

In [0]:
len(vaex_data)

Out[99]: 16186386

In [0]:
# vaex_data = vaex.from_pandas(pandas_data)

vaex_benchmarks = {
    'duration': [],  # in seconds
    'task': [],
}
# vaex_data.head()

In [0]:
# Read file (assuming Parquet format)
def read_file_parquet():
    return vaex.open("/FileStore/tables/yellow_tripdata_2023_01.parquet")

# Count rows
def count(df):
    return len(df)

# Count index length
def count_index_length(df):
    return len(df)

# Mean
def mean(df):
    return df['fare_amount'].mean()

# Standard deviation
def standard_deviation(df):
    return df['fare_amount'].std()

# Mean of sum
def mean_of_sum(df):
    return (df['fare_amount'] + df['tip_amount']).mean()

# Sum columns
def sum_columns(df):
    return (df['fare_amount'] + df['tip_amount']).sum()

# Mean of product
def mean_of_product(df):
    return (df['fare_amount'] * df['tip_amount']).mean()

# Product columns
def product_columns(df):
    return (df['fare_amount'] * df['tip_amount']).sum()

# Value counts
def value_counts(df):
    return df['fare_amount'].value_counts()

# Mean of complicated arithmetic operation
def mean_of_complicated_arithmetic_operation(df):
    theta_1 = df['start_lon']
    phi_1 = df['start_lat']
    theta_2 = df['end_lon']
    phi_2 = df['end_lat']
    temp = (vaex.utils.deg2rad(theta_2 - theta_1) / 2).sin() ** 2 + (vaex.utils.deg2rad(phi_2 - phi_1) / 2).sin() ** 2
    ret = (2 * vaex.utils.rad2deg(temp.sqrt() / (1 - temp).sqrt())).mean()
    return ret

# Complicated arithmetic operation
def complicated_arithmetic_operation(df):
    theta_1 = df['start_lon']
    phi_1 = df['start_lat']
    theta_2 = df['end_lon']
    phi_2 = df['end_lat']
    temp = (vaex.utils.deg2rad(theta_2 - theta_1) / 2).sin() ** 2 + (vaex.utils.deg2rad(phi_2 - phi_1) / 2).sin() ** 2
    ret = 2 * vaex.utils.rad2deg(vaex.utils.arctan2(temp.sqrt(), (1 - temp).sqrt()))
    return ret

# Groupby statistics
def groupby_statistics(df):
    return df.groupby(by=df['passenger_count']).agg({
        'fare_amount': ['mean', 'std'],
        'tip_amount': ['mean', 'std']
    })

other = groupby_statistics(vaex_data)

# Join count
def join_count(df, other):
    return len(df.join(other, on='passenger_count', rsuffix='_other'))

# Join data
def join_data(df, other):
    return df.join(other, on='passenger_count', rsuffix='_other')

In [0]:
#benchmark(read_file_parquet, df=None, benchmarks=dask_benchmarks, name='read file')
benchmark(count, df=vaex_data, benchmarks=vaex_benchmarks, name='count')
benchmark(count_index_length, df=vaex_data, benchmarks=vaex_benchmarks, name='count index length')
benchmark(mean, df=vaex_data, benchmarks=vaex_benchmarks, name='mean')
benchmark(standard_deviation, df=vaex_data, benchmarks=vaex_benchmarks, name='standard deviation')
benchmark(mean_of_sum, df=vaex_data, benchmarks=vaex_benchmarks, name='mean of columns addition')
benchmark(sum_columns, df=vaex_data, benchmarks=vaex_benchmarks, name='addition of columns')
benchmark(mean_of_product, df=vaex_data, benchmarks=vaex_benchmarks, name='mean of columns multiplication')
benchmark(product_columns, df=vaex_data, benchmarks=vaex_benchmarks, name='multiplication of columns')
benchmark(value_counts, df=vaex_data, benchmarks=vaex_benchmarks, name='value counts')
# No column for this
# benchmark(mean_of_complicated_arithmetic_operation, df=dask_data, benchmarks=dask_benchmarks, name='mean of complex arithmetic ops')
# benchmark(complicated_arithmetic_operation, df=dask_data, benchmarks=dask_benchmarks, name='complex arithmetic ops')
benchmark(groupby_statistics, df=vaex_data, benchmarks=vaex_benchmarks, name='groupby statistics')
benchmark(join_count, vaex_data, benchmarks=vaex_benchmarks, name='join count', other=other)
benchmark(join_data, vaex_data, benchmarks=vaex_benchmarks, name='join', other=other) # cant join

count took: 2.193450927734375e-05 seconds
count index length took: 1.0013580322265625e-05 seconds
mean took: 0.08758163452148438 seconds
standard deviation took: 0.6647777557373047 seconds
mean of columns addition took: 0.1370091438293457 seconds
addition of columns took: 0.1096642017364502 seconds
mean of columns multiplication took: 0.14014101028442383 seconds
multiplication of columns took: 0.11284995079040527 seconds
value counts took: 0.3455171585083008 seconds
groupby statistics took: 1.5505948066711426 seconds
join count took: 0.25740551948547363 seconds
join took: 0.23580574989318848 seconds
Out[128]: 0.23580574989318848

Operations with filtering

In [0]:
expr_filter = (vaex_data.tip_amount >= 1) & (vaex_data.tip_amount <= 5)

def filter_data(df):
    return df[expr_filter]
  
vaex_filtered = filter_data(vaex_data)

In [0]:
benchmark(count, vaex_filtered, benchmarks=vaex_benchmarks, name='filtered count')
benchmark(count_index_length, vaex_filtered, benchmarks=vaex_benchmarks, name='filtered count index length')
benchmark(mean, vaex_filtered, benchmarks=vaex_benchmarks, name='filtered mean')
benchmark(standard_deviation, vaex_filtered, benchmarks=vaex_benchmarks, name='filtered standard deviation')
benchmark(mean_of_sum, vaex_filtered, benchmarks=vaex_benchmarks, name ='filtered mean of columns addition')
benchmark(sum_columns, df=vaex_filtered, benchmarks=vaex_benchmarks, name='filtered addition of columns')
benchmark(mean_of_product, vaex_filtered, benchmarks=vaex_benchmarks, name ='filtered mean of columns multiplication')
benchmark(product_columns, df=vaex_filtered, benchmarks=vaex_benchmarks, name='filtered multiplication of columns')
#benchmark(mean_of_complicated_arithmetic_operation, dask_filtered, benchmarks=dask_benchmarks, name='filtered mean of complex arithmetic ops')
#benchmark(complicated_arithmetic_operation, dask_filtered, benchmarks=dask_benchmarks, name='filtered complex arithmetic ops')
benchmark(value_counts, vaex_filtered, benchmarks=vaex_benchmarks, name ='filtered value counts')
benchmark(groupby_statistics, vaex_filtered, benchmarks=vaex_benchmarks, name='filtered groupby statistics')

other = groupby_statistics(vaex_filtered)
# other.columns = pd.Index([e[0]+'_' + e[1] for e in other.columns.tolist()])

benchmark(join_count, vaex_filtered, benchmarks=vaex_benchmarks, name='filtered join count', other=other)
benchmark(join_data, vaex_filtered, benchmarks=vaex_benchmarks, name='filtered join', other=other)

filtered count took: 0.10258865356445312 seconds
filtered count index length took: 1.7881393432617188e-05 seconds
filtered mean took: 0.14691710472106934 seconds
filtered standard deviation took: 0.4904634952545166 seconds
filtered mean of columns addition took: 0.3168008327484131 seconds
filtered addition of columns took: 0.27278733253479004 seconds
filtered mean of columns multiplication took: 0.2804408073425293 seconds
filtered multiplication of columns took: 0.25301241874694824 seconds
filtered value counts took: 0.31861042976379395 seconds
filtered groupby statistics took: 1.5305016040802002 seconds
filtered join count took: 0.23292326927185059 seconds
filtered join took: 0.2331104278564453 seconds
Out[130]: 0.2331104278564453

In [0]:
vaex_res_temp = get_results(vaex_benchmarks).set_index('task')
vaex_res_temp

,duration
task,
count,0.000022
count index length,0.000010
mean,0.087582
standard deviation,0.664778
mean of columns addition,0.137009
addition of columns,0.109664
mean of columns multiplication,0.140141
multiplication of columns,0.112850
value counts,0.345517
